# Competition Math 数据集分析

## 简介

本notebook用于分析从Hugging Face下载的competition_math数据集。该数据集包含来自数学竞赛的问题，如AMC 10、AMC 12、AIME等，每个问题都有完整的逐步解答。

数据集来源: https://huggingface.co/datasets/qwedsacf/competition_math

In [ ]:
# 导入必要的库
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pyarrow.parquet as pq

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置图形样式
sns.set_style("whitegrid")
%matplotlib inline

## 数据加载

让我们加载competition_math数据集并查看其基本结构。

In [ ]:
# 定义数据集路径
dataset_path = "/Users/jia/datasets/competition_math"
parquet_file = os.path.join(dataset_path, "data", "train-00000-of-00001-7320a6f3aba8ebd2.parquet")

# 检查数据集路径是否存在
print("=== 数据集信息 ===")
print(f"数据集路径: {dataset_path}")
print(f"路径是否存在: {os.path.exists(dataset_path)}")
print(f"Parquet文件是否存在: {os.path.exists(parquet_file)}")

# 加载数据
print("\n=== 加载数据 ===")
table = pq.read_table(parquet_file)
df = table.to_pandas()

print(f"数据集形状: {df.shape}")
print(f"列名: {list(df.columns)}")
print(f"数据类型:\n{df.dtypes}")

## 数据概览

现在让我们查看数据的基本统计信息和前几行记录。

In [ ]:
# 显示数据集基本信息
print("=== 数据集基本信息 ===")
print(f"总记录数: {len(df)}")
print(f"列数: {len(df.columns)}")
print(f"缺失值:\n{df.isnull().sum()}")

# 显示前5行数据
print("\n=== 前5行数据 ===")
df.head()

## 统计分析

让我们对数据集进行详细的统计分析，包括问题类型分布和难度等级分布。

In [ ]:
# 问题类型分布
print("=== 问题类型分布 ===")
type_counts = df['type'].value_counts()
type_percentages = df['type'].value_counts(normalize=True) * 100

type_distribution = pd.DataFrame({
    '数量': type_counts,
    '百分比': type_percentages
})
print(type_distribution)

# 难度等级分布
print("\n=== 难度等级分布 ===")
level_counts = df['level'].value_counts()
level_percentages = df['level'].value_counts(normalize=True) * 100

level_distribution = pd.DataFrame({
    '数量': level_counts,
    '百分比': level_percentages
})
print(level_distribution)

# 问题类型和难度等级交叉分析
print("\n=== 问题类型和难度等级交叉分布 ===")
cross_tab = pd.crosstab(df['type'], df['level'], margins=True)
print(cross_tab)

## 数据可视化

通过图表更直观地展示数据分布。

In [ ]:
# 创建图表
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Competition Math 数据集分析', fontsize=16)

# 问题类型分布饼图
axes[0, 0].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('问题类型分布')

# 难度等级分布柱状图
axes[0, 1].bar(level_counts.index, level_counts.values, color='skyblue')
axes[0, 1].set_title('难度等级分布')
axes[0, 1].set_xlabel('难度等级')
axes[0, 1].set_ylabel('问题数量')

# 问题类型分布柱状图
axes[1, 0].barh(type_counts.index, type_counts.values, color='lightgreen')
axes[1, 0].set_title('问题类型分布')
axes[1, 0].set_xlabel('问题数量')
axes[1, 0].set_ylabel('问题类型')

# 热力图显示类型和难度的交叉分布
cross_tab_no_margins = pd.crosstab(df['type'], df['level'])
sns.heatmap(cross_tab_no_margins, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1])
axes[1, 1].set_title('问题类型和难度等级交叉分布')
axes[1, 1].set_xlabel('难度等级')
axes[1, 1].set_ylabel('问题类型')

plt.tight_layout()
plt.show()

## 详细样本展示

现在让我们查看一些具体的问题和解答样本。

In [ ]:
def show_samples(df, n=3):
    """
    显示数据集中的样本
    """
    print(f"=== 显示 {n} 个样本 ===")
    
    # 随机选择样本
    samples = df.sample(n=n, random_state=42)
    
    for i, (index, row) in enumerate(samples.iterrows(), 1):
        print(f"\n--- 样本 {i} (索引: {index}) ---")
        print(f"问题类型: {row['type']}")
        print(f"难度等级: {row['level']}")
        print(f"问题: {row['problem']}")
        print(f"解答: {row['solution']}")
        print("-" * 50)

# 显示样本
show_samples(df, 3)

## 按类型查看样本

让我们按不同类型查看一些样本。

In [ ]:
def show_type_samples(df, problem_type, n=2):
    """
    按问题类型显示样本
    """
    print(f"=== {problem_type} 类型样本 ===")
    
    # 筛选特定类型的问题
    type_df = df[df['type'] == problem_type]
    
    # 选择不同难度等级的样本
    samples = []
    for level in sorted(type_df['level'].unique()):
        level_samples = type_df[type_df['level'] == level]
        if not level_samples.empty:
            sample = level_samples.sample(n=1, random_state=42).iloc[0]
            samples.append(sample)
            if len(samples) >= n:
                break
    
    for i, sample in enumerate(samples, 1):
        print(f"\n--- 样本 {i} ---")
        print(f"难度等级: {sample['level']}")
        print(f"问题: {sample['problem']}")
        print(f"解答: {sample['solution']}")
        print("-" * 50)

# 显示不同类型的问题样本
for problem_type in ['Algebra', 'Geometry', 'Number Theory']:
    show_type_samples(df, problem_type, 2)
    print()
    print()

## 按难度查看样本

让我们按不同难度等级查看一些样本。

In [ ]:
def show_level_samples(df, level, n=2):
    """
    按难度等级显示样本
    """
    print(f"=== {level} 难度等级样本 ===")
    
    # 筛选特定难度等级的问题
    level_df = df[df['level'] == level]
    
    # 选择不同类型的样本
    samples = []
    for problem_type in level_df['type'].unique()[:n]:
        type_samples = level_df[level_df['type'] == problem_type]
        if not type_samples.empty:
            sample = type_samples.sample(n=1, random_state=42).iloc[0]
            samples.append(sample)
    
    for i, sample in enumerate(samples, 1):
        print(f"\n--- 样本 {i} ---")
        print(f"问题类型: {sample['type']}")
        print(f"问题: {sample['problem']}")
        print(f"解答: {sample['solution']}")
        print("-" * 50)

# 显示不同难度等级的问题样本
for level in ['Level 1', 'Level 3', 'Level 5']:
    show_level_samples(df, level, 2)
    print()
    print()

## 文本分析

让我们对问题和解答的文本进行一些基本分析。

In [ ]:
# 计算问题和解答的长度
df['problem_length'] = df['problem'].str.len()
df['solution_length'] = df['solution'].str.len()

print("=== 文本长度统计 ===")
print("问题长度统计:")
print(df['problem_length'].describe())
print("\n解答长度统计:")
print(df['solution_length'].describe())

# 按类型分析文本长度
print("\n=== 按问题类型分析文本长度 ===")
type_length_stats = df.groupby('type')[['problem_length', 'solution_length']].mean()
print(type_length_stats)

# 按难度等级分析文本长度
print("\n=== 按难度等级分析文本长度 ===")
level_length_stats = df.groupby('level')[['problem_length', 'solution_length']].mean()
print(level_length_stats)

## 总结

### 数据集概览

1. **数据规模**: competition_math数据集包含12,500个数学问题
2. **字段信息**: 
   - problem: 数学问题描述
   - level: 难度等级(Level 1-5)
   - type: 问题类型(Algebra, Geometry等)
   - solution: 详细解题过程

### 问题类型分布
1. **代数 (Algebra)**: 2,931个问题(23.4%)
2. **中等代数 (Intermediate Algebra)**: 2,198个问题(17.6%)
3. **基础代数 (Prealgebra)**: 2,076个问题(16.6%)
4. **数论 (Number Theory)**: 1,409个问题(11.3%)
5. **几何 (Geometry)**: 1,349个问题(10.8%)
6. **预微积分 (Precalculus)**: 1,292个问题(10.3%)
7. **计数与概率 (Counting & Probability)**: 1,245个问题(10.0%)

### 难度等级分布
1. **Level 5**: 3,628个问题(29.0%)
2. **Level 4**: 2,904个问题(23.2%)
3. **Level 3**: 2,723个问题(21.8%)
4. **Level 2**: 2,242个问题(17.9%)
5. **Level 1**: 1,001个问题(8.0%)

### 数据特点
1. **高质量数据**: 每个问题都有完整的逐步解答
2. **多样性**: 涵盖多个数学领域和不同难度等级
3. **教育价值**: 适合用于训练数学问题解决AI模型
4. **研究价值**: 可用于评估AI模型的数学推理能力

这个数据集非常适合用于训练和评估需要复杂数学推理能力的AI模型。